In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorclan
import tensorclan.dataset as tcd
import tensorclan.model as tcm
import tensorclan.utils as tc_utils
import tensorclan.runner as tc_runner

In [3]:
%%writefile custom_model.yaml

name: TTC_EXAMPLE
log_dir: logs
chkpt_dir: checkpoint
use_checkpoints: false

device: GPU

model:
    name: Net

dataset:
    name: MNIST
    transforms: MNISTTransforms
    root: data
    loader_args:
        batch_size: 128
        num_workers: 2
        shuffle: True
        pin_memory: True

loss: CrossEntropyLoss

optimizer:
    type: SGD
    args:
        lr: 0.01
        momentum: 0.95

training:
    epochs: 10

Overwriting custom_model.yaml


In [4]:
@tcm.model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

[ 2020-07-25 15:18:50,464 - tensorclan.model.model ] INFO: Registered Net as model


In [5]:
config = tc_utils.load_config('custom_model.yaml')

In [6]:
runner = tc_runner.Runner(config=config)


[ 2020-07-25 15:18:50,504 - tensorclan.runner.runner ] INFO: => Now simply setup_train and then start_train your model


In [7]:
runner.setup_train()

[ 2020-07-25 15:18:50,520 - tensorclan.runner.runner ] INFO: => Config
[ 2020-07-25 15:18:50,521 - tensorclan.runner.runner ] INFO: {'chkpt_dir': 'checkpoint',
[ 2020-07-25 15:18:50,521 - tensorclan.runner.runner ] INFO:  'dataset': {'loader_args': {'batch_size': 128,
[ 2020-07-25 15:18:50,521 - tensorclan.runner.runner ] INFO:                              'num_workers': 2,
[ 2020-07-25 15:18:50,522 - tensorclan.runner.runner ] INFO:                              'pin_memory': True,
[ 2020-07-25 15:18:50,522 - tensorclan.runner.runner ] INFO:                              'shuffle': True},
[ 2020-07-25 15:18:50,522 - tensorclan.runner.runner ] INFO:              'name': 'MNIST',
[ 2020-07-25 15:18:50,523 - tensorclan.runner.runner ] INFO:              'root': 'data',
[ 2020-07-25 15:18:50,523 - tensorclan.runner.runner ] INFO:              'transforms': 'MNISTTransforms'},
[ 2020-07-25 15:18:50,523 - tensorclan.runner.runner ] INFO:  'device': 'GPU',
[ 2020-07-25 15:18:50,523 - tensorcla

In [8]:
runner.start_train()

[ 2020-07-25 15:18:52,072 - tensorclan.trainer.gpu_trainer ] INFO: => Training Started
[ 2020-07-25 15:18:52,073 - tensorclan.trainer.gpu_trainer ] INFO: Training the model for 10 epochs
[ 2020-07-25 15:18:52,074 - tensorclan.trainer.gpu_trainer ] INFO: => Training Epoch 0
[ 2020-07-25 15:18:56,277 - tensorclan.trainer.gpu_trainer ] INFO: loss: 0.6926954992290245, accuracy: 77.08333333333333
[ 2020-07-25 15:18:56,278 - tensorclan.trainer.gpu_trainer ] INFO: => Testing Epoch 0
[ 2020-07-25 15:18:56,996 - tensorclan.trainer.gpu_trainer ] INFO: loss: 0.10924185086277467 accuracy: 96.67
[ 2020-07-25 15:18:56,998 - tensorclan.trainer.gpu_trainer ] INFO: => Training Epoch 1
[ 2020-07-25 15:19:00,853 - tensorclan.trainer.gpu_trainer ] INFO: loss: 0.27480914462794626, accuracy: 91.89
[ 2020-07-25 15:19:00,854 - tensorclan.trainer.gpu_trainer ] INFO: => Testing Epoch 1
[ 2020-07-25 15:19:01,558 - tensorclan.trainer.gpu_trainer ] INFO: loss: 0.07733765697177453 accuracy: 97.58
[ 2020-07-25 15:19